# Mod 3 Final Project

Jeremy Owens, Georgina Sampson

In [1]:
import pandas as pd
import sqlite3
import numpy as np
from api_keys import jo_api_key, gs_api_key
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
crime_data = pd.read_csv('denver_data/crime.csv')
dispensary_data = pd.read_csv('denver_data/marijuana_active_business_licenses.csv')
# offense_data = pd.read_csv('denver_data/offense_codes.csv')

In [38]:
print(len(crime_data))
crime_data.sample(10)

496009


,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
115278,2015280793,2015280793544100,5441,0,traffic-accident,traffic-accident,5/22/2015 11:55:00 AM,NaN,5/22/2015 12:19:00 PM,2100 BLOCK N BROADWAY ST,3144159.0,1698908.0,-104.987299,39.751191,6,621,five-points,0,1
361511,20164612,20164612299901,2999,1,criminal-mischief-mtr-veh,public-disorder,1/3/2016 5:00:00 AM,1/3/2016 5:30:00 AM,1/3/2016 5:57:00 AM,3045 S DECATUR ST,3134174.0,1666217.0,-105.023420,39.661596,4,422,college-view-south-platte,1,0
482541,2018787971,2018787971299900,2999,0,criminal-mischief-other,public-disorder,11/19/2018 12:45:00 PM,NaN,11/19/2018 1:48:00 PM,7700 W QUINCY AVE,3117421.0,1656995.0,-105.083076,39.636505,4,423,marston,1,0
336595,2016821670,2016821670239900,2399,0,theft-other,larceny,12/27/2016 11:02:00 AM,12/27/2016 11:07:00 AM,12/27/2016 2:27:00 PM,4845 E THRILL PL,3160080.0,1703425.0,-104.930577,39.763330,2,221,northeast-park-hill,1,0
309408,2015445907,2015445907220300,2203,0,burglary-business-by-force,burglary,8/5/2015 8:00:00 PM,8/6/2015 6:39:00 AM,8/6/2015 6:39:00 AM,1100 N YUMA CT,3136556.0,1692610.0,-105.014459,39.734016,1,123,lincoln-park,1,0
143217,2016341027,2016341027239900,2399,0,theft-other,larceny,5/25/2016 12:00:00 PM,5/31/2016 5:00:00 PM,6/1/2016 1:00:00 AM,1243 N GAYLORD ST,3151480.0,1693384.0,-104.961379,39.735910,6,622,cheesman-park,1,0
215593,20176001111,20176001111230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,2/2/2017 1:55:00 PM,2/2/2017 2:00:00 PM,2/2/2017 3:14:00 PM,500 S SANTA FE DR,3142346.0,1682796.0,-104.994066,39.706988,3,312,baker,1,0
102210,201461580,201461580544100,5441,0,traffic-accident,traffic-accident,2/5/2014 8:55:00 AM,NaN,2/5/2014 9:37:00 AM,2100 BLOCK S SANTA FE DR,3142598.0,1671732.0,-104.993390,39.676611,3,313,overland,0,1
325375,2016587039,2016587039521201,5212,1,weapon-by-prev-offender-powpo,all-other-crimes,9/13/2016 1:20:00 AM,NaN,9/13/2016 4:31:00 AM,4700 BLK N FEDERAL BLVD,3133321.0,1710640.0,-105.025626,39.783558,1,111,berkeley,1,0
403723,2014435555,2014435555549900,5499,0,traf-other,all-other-crimes,8/19/2014 10:45:00 AM,NaN,8/20/2014 4:24:00 AM,3000 BLOCK W 49TH AVE,3133267.0,1711362.0,-105.025805,39.785541,1,111,regis,1,0


In [39]:
crime_data.columns = [x.lower() for x in crime_data.columns]
crime_data.drop(['offense_id', 'offense_code_extension', 'last_occurrence_date', 'incident_address', 
                 'geo_x', 'geo_y', 'geo_lon', 'geo_lat', 'district_id', 'precinct_id'], axis=1, inplace=True)
crime_data.sample(10)

,incident_id,offense_code,offense_type_id,offense_category_id,first_occurrence_date,reported_date,neighborhood_id,is_crime,is_traffic
465641,2017828795,5441,traffic-accident,traffic-accident,12/12/2017 7:50:00 AM,12/12/2017 9:03:00 AM,hampden-south,0,1
185696,201674717,5499,traf-other,all-other-crimes,2/4/2016 10:54:00 PM,2/4/2016 11:16:00 PM,stapleton,1,0
292651,2014426462,5441,traffic-accident,traffic-accident,8/14/2014 5:52:00 PM,8/14/2014 7:33:00 PM,washington-park-west,0,1
50259,2014151493,4104,liquor-possession,drug-alcohol,3/25/2014 12:15:00 PM,3/25/2014 12:29:00 PM,chaffee-park,1,0
46756,2019376163,5499,traf-other,all-other-crimes,6/16/2019 11:10:00 AM,6/16/2019 11:10:00 AM,globeville,1,0
205155,2017344114,1313,assault-dv,other-crimes-against-persons,5/27/2017 11:15:00 AM,5/27/2017 1:06:00 PM,regis,1,0
494395,2018727821,1315,aggravated-assault,aggravated-assault,10/24/2018 1:43:00 PM,10/24/2018 2:18:00 PM,five-points,1,0
244691,2018272133,2303,theft-shoplift,larceny,4/23/2018 6:16:00 PM,4/23/2018 7:07:00 PM,overland,1,0
9044,2018212505,5401,traffic-accident-hit-and-run,traffic-accident,3/31/2018 2:00:00 AM,3/31/2018 2:00:00 AM,baker,0,1
120097,20156004850,2305,theft-items-from-vehicle,theft-from-motor-vehicle,6/2/2015 7:30:00 PM,6/3/2015 11:40:00 AM,speer,1,0


In [40]:
print(len(crime_data))
crime_data.isna().sum()

496009


incident_id              0
offense_code             0
offense_type_id          0
offense_category_id      0
first_occurrence_date    0
reported_date            0
neighborhood_id          0
is_crime                 0
is_traffic               0
dtype: int64

In [41]:
crime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496009 entries, 0 to 496008
Data columns (total 9 columns):
incident_id              496009 non-null int64
offense_code             496009 non-null int64
offense_type_id          496009 non-null object
offense_category_id      496009 non-null object
first_occurrence_date    496009 non-null object
reported_date            496009 non-null object
neighborhood_id          496009 non-null object
is_crime                 496009 non-null int64
is_traffic               496009 non-null int64
dtypes: int64(4), object(5)
memory usage: 34.1+ MB


In [55]:
crime_data = crime_data[crime_data.is_crime == 1]
crime_data.drop(['is_crime', 'is_traffic'], axis=1, inplace=True)

AttributeError: 'DataFrame' object has no attribute 'is_crime'

In [56]:
print(len(crime_data))
crime_data.sample(10)

366991


,incident_id,offense_code,offense_type_id,offense_category_id,first_occurrence_date,reported_date,neighborhood_id
103272,2014681988,5312,disturbing-the-peace,public-disorder,12/26/2014 12:00:00 AM,12/26/2014 12:59:00 AM,congress-park
255944,2018301316,2399,theft-other,larceny,5/5/2018 9:00:00 AM,5/5/2018 3:51:00 PM,elyria-swansea
409251,2014124899,2204,burglary-residence-no-force,burglary,3/11/2014 11:15:00 AM,3/11/2014 4:31:00 PM,mar-lee
345288,201848208,5499,traf-other,all-other-crimes,1/20/2018 6:09:00 PM,1/20/2018 7:23:00 PM,kennedy
171976,2016500066,2303,theft-shoplift,larceny,8/7/2016 3:15:00 PM,8/7/2016 3:23:00 PM,college-view-south-platte
487770,20186011988,2399,theft-other,larceny,12/7/2018 12:35:00 PM,12/9/2018 4:43:00 PM,dia
408279,2014677313,1315,aggravated-assault,aggravated-assault,12/22/2014 8:20:00 PM,12/22/2014 11:38:00 PM,westwood
493577,2018827338,2303,theft-shoplift,larceny,12/7/2018 2:53:00 PM,12/7/2018 3:19:00 PM,virginia-village
231699,2017464182,5016,violation-of-court-order,all-other-crimes,7/13/2017 3:00:00 PM,7/14/2017 7:33:00 AM,montbello
183965,2017249550,5499,traf-other,all-other-crimes,4/17/2017 2:15:00 PM,4/17/2017 2:43:00 PM,baker


In [54]:
list(crime_data.groupby('offense_type_id').count().index)

['accessory-conspiracy-to-crime',
 'agg-aslt-police-weapon',
 'aggravated-assault',
 'aggravated-assault-dv',
 'altering-vin-number',
 'animal-cruelty-to',
 'animal-poss-of-dangerous',
 'arson-business',
 'arson-other',
 'arson-public-building',
 'arson-residence',
 'arson-vehicle',
 'aslt-agg-police-gun',
 'assault-dv',
 'assault-police-simple',
 'assault-simple',
 'bomb-threat',
 'bribery',
 'burg-auto-theft-busn-no-force',
 'burg-auto-theft-busn-w-force',
 'burg-auto-theft-resd-no-force',
 'burg-auto-theft-resd-w-force',
 'burglary-business-by-force',
 'burglary-business-no-force',
 'burglary-poss-of-tools',
 'burglary-residence-by-force',
 'burglary-residence-no-force',
 'burglary-safe',
 'burglary-vending-machine',
 'contraband-into-prison',
 'contraband-possession',
 'criminal-mischief-graffiti',
 'criminal-mischief-mtr-veh',
 'criminal-mischief-other',
 'criminal-trespassing',
 'curfew',
 'disarming-a-peace-officer',
 'disturbing-the-peace',
 'drug-barbiturate-mfr',
 'drug-barbi

In [21]:
offense_data.head(10)

,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_TYPE_NAME,OFFENSE_CATEGORY_ID,OFFENSE_CATEGORY_NAME,IS_CRIME,IS_TRAFFIC
0,2804,1,stolen-property-possession,Possession of stolen property,all-other-crimes,All Other Crimes,1,0
1,2804,2,fraud-possess-financial-device,Possession of a financial device,all-other-crimes,All Other Crimes,1,0
2,2901,0,damaged-prop-bus,Damaged business property,public-disorder,Public Disorder,1,0
3,2902,0,criminal-mischief-private,Criminal mischief to private property,public-disorder,Public Disorder,1,0
4,2903,0,criminal-mischief-public,Criminal mischief to public property,public-disorder,Public Disorder,1,0
5,2999,0,criminal-mischief-other,Criminal mischief - other,public-disorder,Public Disorder,1,0
6,2999,1,criminal-mischief-mtr-veh,Criminal mischief to a motor vehicle,public-disorder,Public Disorder,1,0
7,2999,2,criminal-mischief-graffiti,Criminal mischief - graffiti,public-disorder,Public Disorder,1,0
8,3501,0,drug-hallucinogen-mfr,Manufacture of a hallucinogenic drug,drug-alcohol,Drug & Alcohol,1,0
9,3503,0,drug-hallucinogen-sell,Selling a hallucinogenic drug,drug-alcohol,Drug & Alcohol,1,0


In [25]:
offense_data.groupby('OFFENSE_CATEGORY_ID')['OFFENSE_CODE'].count()

OFFENSE_CATEGORY_ID
aggravated-assault               13
all-other-crimes                107
arson                             5
auto-theft                        4
burglary                         11
drug-alcohol                     30
larceny                          15
murder                            9
other-crimes-against-persons     14
public-disorder                  27
robbery                          13
sexual-assault                   13
theft-from-motor-vehicle          3
traffic-accident                  6
white-collar-crime               29
Name: OFFENSE_CODE, dtype: int64

In [9]:
print(len(dispensary_data))
dispensary_data.sample(10)

984


,Business File Number,License Type,Entity Name,Trade Name,Current License Status,Expiration Date,Facility Street Number,Facility Pre-Direction,Facility Street Name,Facility Street Type,Facility Unit Number,Facility City,Facility Zip Code
953,2012-BFN-1060569,Med Marijuana Opt Prem Cultiv,"TWOG-FEDERAL, LLC",BUDDY BOY,License Issued - Active,2/13/2020 12:00:00 AM,5040,N,YORK,ST,NaN,DENVER,80216.0
126,2010-BFN-1045824,Medical Marijuana Center,"DJR COLORADO, LLC",NaN,License Issued - Active,9/10/2019 12:00:00 AM,1620,NaN,MARKET,ST,5W,DENVER,80202.0
697,2014-BFN-1074182,Retail Marijuana Store,"SERENITY MOON WELLNESS CENTER, LLC",FROSTED LEAF,License Issued - Active,9/10/2019 12:00:00 AM,6302,E,COLFAX,AVE,NaN,DENVER,80220.0
194,2017-BFN-0006975,Retail Marijuana Store,"THE GREEN SOLUTION, LLC",NaN,License Issued - Active,1/26/2020 12:00:00 AM,1995,NaN,WEWATTA,ST,NaN,DENVER,80202.0
859,2017-BFN-0002518,Retail Marijuana Inf Prod Mfg,"CONSCIOUS CONFECTIONS II, LLC",NATIVE ROOTS EXTRACTS,License Issued - Active,12/1/2019 12:00:00 AM,4990,N,DAHLIA,ST,NaN,DENVER,80216.0
872,2010-BFN-1045897,Medical Marijuana Center,"ABUNDANT HEALTH DENVER, LLC",HERBAN UNDERGROUND,License Issued - Active,9/11/2019 12:00:00 AM,70,N,BROADWAY,NaN,50,DENVER,80203.0
384,2013-BFN-1064539,Med Marijuana Inf Prod Manuf,DRAGON ORIGINALS LLC,NaN,License Issued - Active,9/25/2019 12:00:00 AM,4093,N,Jackson,ST,NaN,Denver,80216.0
307,2012-BFN-1061669,Med Marijuana Inf Prod Manuf,"LAB 710, LLC",LAB 710,License Issued - Active,3/21/2020 12:00:00 AM,1346,W,CEDAR,AVE,NaN,DENVER,80223.0
629,2018-BFN-0006220,Retail Marijuana Inf Prod Mfg,PAPILLONS LLC,PAPILLONS,License Issued - Active,5/17/2020 12:00:00 AM,963,N,Federal,BLVD,120,Denver,NaN
845,2014-BFN-1070939,Retail Marij Opt. Prem. Cultiv,"VIOLA, INC",VIOLA'S EXTRACTS,License Issued - Active,4/25/2020 12:00:00 AM,4645,N,Joliet,NaN,NaN,Denver,80239.0


In [28]:
dispensary_data.groupby('Trade Name').count()

,Business File Number,License Type,Entity Name,Current License Status,Expiration Date,Facility Street Number,Facility Pre-Direction,Facility Street Name,Facility Street Type,Facility Unit Number,Facility City,Facility Zip Code
Trade Name,,,,,,,,,,,,
.,1,1,1,1,1,1,1,1,1,0,1,1
1136 YUMA,5,5,5,5,5,5,5,5,5,0,5,5
5 BUDS,1,1,1,1,1,1,1,1,0,0,1,1
5280 WEEDERY LLC,3,3,3,3,3,3,3,3,3,0,3,3
710 LABS,2,2,2,2,2,2,2,2,2,0,2,2
A CUT ABOVE,2,2,2,2,2,2,2,2,0,0,2,2
ADAMS GREEN CROSSING,2,2,2,2,2,2,2,2,2,0,2,2
AFFINITY,4,4,4,4,4,4,4,4,4,0,4,4
AGRISCIENCE LABS,2,2,2,2,2,2,2,2,2,0,2,2


In [29]:
dispensary_data[dispensary_data['Trade Name'] == '1136 YUMA']

,Business File Number,License Type,Entity Name,Trade Name,Current License Status,Expiration Date,Facility Street Number,Facility Pre-Direction,Facility Street Name,Facility Street Type,Facility Unit Number,Facility City,Facility Zip Code
0,2016-BFN-0007402,Retail Marijuana Store,"YUMA WAY, LLC",1136 YUMA,License Issued - Active,11/6/2019 12:00:00 AM,1136,N,YUMA,CT,NaN,DENVER,80204.0
303,2015-BFN-0007653,Med Marijuana Opt Prem Cultiv,YUMA WAY LLC,1136 YUMA,License Issued - Active,4/13/2020 12:00:00 AM,1240,S,Bannock,ST,NaN,Denver,80223.0
542,2015-BFN-0007808,Med Marijuana Opt Prem Cultiv,YUMA WAY LLC,1136 YUMA,License Issued - Active,4/13/2020 12:00:00 AM,1136,N,YUMA,CT,NaN,DENVER,80204.0
790,2016-BFN-0004407,Retail Marij Opt. Prem. Cultiv,"YUMA WAY, LLC",1136 YUMA,License Issued - Active,9/22/2019 12:00:00 AM,1136,N,YUMA,CT,NaN,DENVER,80204.0
807,2016-BFN-0008405,Retail Marij Opt. Prem. Cultiv,"YUMA WAY, LLC",1136 YUMA,License Issued - Active,2/3/2020 12:00:00 AM,1240,S,Bannock,St,NaN,Denver,80223.0
